In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from tianchi.user_location_prediction.pycode.load_data import *
from tianchi.user_location_prediction.pycode.gen_train_test_data import gen_train_test_data
import time

In [10]:
sample = load_sample()

/Users/zhouzhirui/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [23]:
sample.head(3)

,sample_id,row_id,user_id,mall_id,time,lgt,ltt,wifi_infos,shop_id,len
0,0,NaN,u_376,m_1409,2017-08-06 21:20:00,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...,s_2871718,204
1,1,NaN,u_376,m_1409,2017-08-06 21:20:00,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...,s_2871718,205
2,2,NaN,u_1041,m_4079,2017-08-02 13:10:00,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...,s_181637,205


In [28]:
sample_i = sample[sample.mall_id=='m_1409']

In [33]:
from sklearn.preprocessing import MinMaxScaler

In [34]:
def extract_space_feat(df_):
    df = df_.copy()
    scaler = MinMaxScaler()
    feat = scaler.fit_transform(df[['lgt','ltt']])
    return feat

def extract_time_feat(df_):
    df = df_.copy()
    df['weekday'] = df.time.dt.weekday
    df['hour'] = df.time.dt.hour
    scaler = MinMaxScaler()
    feat = scaler.fit_transform(df[['weekday','hour']])
    return feat

In [36]:
f1 = extract_space_feat(sample_i)
f2 = extract_time_feat(sample_i)

In [39]:
sample_i.head()

,sample_id,row_id,user_id,mall_id,time,lgt,ltt,wifi_infos,shop_id,len
0,0,NaN,u_376,m_1409,2017-08-06 21:20:00,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...,s_2871718,204
1,1,NaN,u_376,m_1409,2017-08-06 21:20:00,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...,s_2871718,205
68,68,NaN,u_13782,m_1409,2017-08-14 20:40:00,122.309295,32.088638,b_11592648|-74|false;b_56326068|-61|false;b_29...,s_52283,208
80,80,NaN,u_18171,m_1409,2017-08-11 15:30:00,122.310365,32.087681,b_41870824|-84|false;b_51825273|-76|false;b_41...,s_288430,209
156,156,NaN,u_29343,m_1409,2017-08-14 13:30:00,122.310139,32.087676,b_55113661|-76|false;b_52642831|-73|false;b_38...,s_288430,209


In [40]:
def extract_wifi_feat(df):
    sample_id_list = []
    wifi_id_list = []
    signal_power_list = []
    signal_flag_list = []

    df = df.reset_index(drop=True)
    sample_size = df.shape[0]

    for i in range(sample_size):
        wifi_info = df.wifi_infos[i].split(';')
        sample_id = df.sample_id[i]
        for w in wifi_info:
            w_values = w.split('|')
            wifi_id_list.append(w_values[0])
            signal_power_list.append(float(w_values[1]))
            signal_flag_list.append(w_values[2])
            sample_id_list.append(sample_id)

    feat = pd.DataFrame({'sample_id': sample_id_list,
                         'signal_power': signal_power_list,
                         'signal_flag': signal_flag_list,
                         'wifi_id': wifi_id_list},
                        columns=['sample_id', 'wifi_id', 'signal_power', 'signal_flag'])
    feat['signal_flag'] = feat.signal_flag.apply(lambda x: 1 if x == 'true' else 0)

    wifi_power = pd.pivot_table(feat, index='sample_id', columns='wifi_id', values='signal_power')
    wifi_flag = pd.pivot_table(feat, index='sample_id', columns='wifi_id', values='signal_flag')
    wifi_feat = pd.concat([wifi_power, wifi_flag], axis=1).reset_index().fillna(0)
    return wifi_feat

In [41]:
f3 = extract_wifi_feat(sample_i)

In [42]:
f3.shape

(23608, 14185)

In [43]:
f3

wifi_id,sample_id,b_10004577,b_10004588,b_10063227,b_10066720,b_10071970,b_10083821,b_1009121,b_1009122,b_10093712,...,b_9761636,b_9809138,b_9837892,b_9893403,b_9926786,b_9935522,b_9964950,b_9972974,b_9983657,b_9986470
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
